# 4. BorderlineSMOTE
###  모든 minority data point가 아니라 Border-line 근처에 있는 관측치들에 대해서 집중적으로 오버샘플링을 실시하는 방법. border line 근처에 있을수록 상대적으로 분류하기가 힘들기 때문에 이러한 데이터 샘플을 추가로 생성하고 학습하여 모델의 성능을 향상시키는 방법. 

In [1]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import BorderlineSMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("./데이터자료/9-2. Std 데이터셋.csv")
feature = pd.read_csv("./데이터자료/8-2. Std feature.csv")
df

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,산업명,매출액증가율,유형자산증가율,재고자산증가율,...,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,label,회계년
0,(주)CMG제약,58820,2011-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,1.186494,-0.286555,-0.242712,...,-0.517775,-0.268715,-0.204982,-0.097181,0.001823,0.167352,-0.034998,-0.128958,0.0,2011
1,(주)CMG제약,58820,2012-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,-0.465320,-0.266139,-0.145700,...,-0.570848,-0.965957,-0.239707,-0.097246,-0.455154,7.331845,-8.395971,-8.842764,0.0,2012
2,(주)CMG제약,58820,2013-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.399443,-0.034331,0.368591,...,-0.459402,-0.280668,-0.160693,-0.097092,0.377872,-0.016097,0.127964,0.136844,0.0,2013
3,(주)CMG제약,58820,2014-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.042989,0.633335,-0.160104,...,-0.458301,-0.300589,-0.202938,-0.097179,0.294901,0.063259,0.057470,0.125748,0.0,2014
4,(주)CMG제약,58820,2015-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.229752,-0.280476,-0.110645,...,-0.503803,-0.558426,-0.217252,-0.097201,-0.090083,0.704439,-0.512105,-0.358349,0.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9273,휴마시스(주),205470,2020-12-01,2014-12-01,NaN,5,의료용 물질 및 의약품 제조업,5.662930,1.347455,2.510801,...,-0.151711,1.140567,-0.125778,-0.096931,0.930074,-0.576239,0.625551,0.569922,0.0,2020
9274,휴젤(주),145020,2019-12-01,2015-12-24,NaN,5,의료용 물질 및 의약품 제조업,0.059296,-0.046260,0.113775,...,1.299954,-0.272699,-0.062404,-0.097006,0.914756,-0.434150,0.499330,0.360994,0.0,2019
9275,휴젤(주),145020,2020-12-01,2015-12-24,NaN,5,의료용 물질 및 의약품 제조업,-0.170016,-0.100971,-0.437478,...,1.349171,-0.301727,-0.058107,-0.096969,0.907353,-0.415350,0.482630,0.330828,0.0,2020
9276,흥구석유(주),24060,2019-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,-0.488761,-0.299172,0.793390,...,-0.392823,-0.500939,-0.145811,-0.095817,-0.354313,-0.106535,0.208302,0.265758,0.0,2019


In [4]:
train = df[df["회계년"] <= 2018].reset_index(drop='index')
test  = df[df["회계년"] > 2018].reset_index(drop='index')
col = df.columns[7:55]

In [5]:
x_train = train[col]
y_train = train["label"]
x_test = test[col]
y_test = test["label"]
print(x_train.shape)
y_train.value_counts()

(6913, 48)


0.0    6792
1.0     121
Name: label, dtype: int64

In [6]:
bdsmt = BorderlineSMOTE()
x_train, y_train = bdsmt.fit_resample(x_train,y_train)
print(x_train.shape)
y_train.value_counts()

(13584, 48)


0.0    6792
1.0    6792
Name: label, dtype: int64

In [7]:
mda_feature=feature["mda_feature"].tolist()
logit_feature=feature["logit_feature"].tolist()
lasso_feature=feature["lasso_feature"].tolist()
ela_feature=feature["ela_feature"].tolist()
same_feature=feature["same_feature"].tolist()

In [8]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [9]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")


In [10]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

In [11]:
mda_x_train = x_train[mda_feature].reset_index(drop='index')
mda_x_test = x_test[mda_feature].reset_index(drop='index')
mda_y_train = y_train
mda_y_test = test["label"]

In [12]:
logit_x_train = x_train[logit_feature].reset_index(drop='index')
logit_x_test = x_test[logit_feature].reset_index(drop='index')
logit_y_train = y_train
logit_y_test = test["label"]

In [13]:
lasso_x_train = x_train[lasso_feature].reset_index(drop='index')
lasso_x_test = x_test[lasso_feature].reset_index(drop='index')
lasso_y_train = y_train
lasso_y_test = test["label"]

In [14]:
ela_x_train = x_train[ela_feature].reset_index(drop='index')
ela_x_test = x_test[ela_feature].reset_index(drop='index')
ela_y_train = y_train
ela_y_test = test["label"]

In [15]:
same_x_train = x_train[same_feature].reset_index(drop='index')
same_x_test = x_test[same_feature].reset_index(drop='index')
same_y_train = y_train
same_y_test = test["label"]

In [16]:
fea_x_train = [mda_x_train, logit_x_train, lasso_x_train, ela_x_train, same_x_train]
fea_y_train = [mda_y_train, logit_y_train, lasso_y_train, ela_y_train, same_y_train]
fea_x_test = [mda_x_test, logit_x_test, lasso_x_test, ela_x_test, same_x_test]
fea_y_test = [mda_y_test, logit_y_test, lasso_y_test, ela_y_test, same_y_test]
fea = ["mda","logit","lasso","ela","same"]

In [17]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    lda_clf.fit(x_train, y_train)
    lda_pred = lda_clf.predict(x_test)
    lda_pred_proba = lda_clf.predict_proba(x_test)
    print(feature,"lda") 
    get_clf_eval(y_test, lda_pred)
    get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)



mda lda
오차행렬
[[2056  259]
 [  19   31]]
정확도: 0.8825, 정밀도 : 0.1069, 재현율:0.6200,F1 스코어:0.1824
ROC 스코어: 0.7541, PR score : 0.0743
임곗값: 0.1
오차행렬
[[1488  827]
 [  12   38]]
정확도: 0.6452, 정밀도 : 0.0439, 재현율:0.7600,F1 스코어:0.0831
ROC 스코어: 0.7014, PR score : 0.0385
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1786  529]
 [  14   36]]
정확도: 0.7704, 정밀도 : 0.0637, 재현율:0.7200,F1 스코어:0.1171
ROC 스코어: 0.7457, PR score : 0.0518
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1912  403]
 [  15   35]]
정확도: 0.8233, 정밀도 : 0.0799, 재현율:0.7000,F1 스코어:0.1434
ROC 스코어: 0.7630, PR score : 0.0623
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1998  317]
 [  16   34]]
정확도: 0.8592, 정밀도 : 0.0969, 재현율:0.6800,F1 스코어:0.1696
ROC 스코어: 0.7715, PR score : 0.0726
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2056  259]
 [  19   31]]
정확도: 0.8825, 정밀도 : 0.1069, 재현율:0.6200,F1 스코어:0.1824
ROC 스코어: 0.7541, PR score : 0.0743
-

In [18]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    dt_clf.fit(x_train, y_train)
    dt_pred = dt_clf.predict(x_test)
    dt_pred_proba = dt_clf.predict_proba(x_test)   
    print(feature,"디시전트리") 
    get_clf_eval(y_test, dt_pred)
    get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


mda 디시전트리
오차행렬
[[2218   97]
 [  31   19]]
정확도: 0.9459, 정밀도 : 0.1638, 재현율:0.3800,F1 스코어:0.2289
ROC 스코어: 0.6690, PR score : 0.0753
임곗값: 0.1
오차행렬
[[2218   97]
 [  31   19]]
정확도: 0.9459, 정밀도 : 0.1638, 재현율:0.3800,F1 스코어:0.2289
ROC 스코어: 0.6690, PR score : 0.0753
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2218   97]
 [  31   19]]
정확도: 0.9459, 정밀도 : 0.1638, 재현율:0.3800,F1 스코어:0.2289
ROC 스코어: 0.6690, PR score : 0.0753
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2218   97]
 [  31   19]]
정확도: 0.9459, 정밀도 : 0.1638, 재현율:0.3800,F1 스코어:0.2289
ROC 스코어: 0.6690, PR score : 0.0753
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2218   97]
 [  31   19]]
정확도: 0.9459, 정밀도 : 0.1638, 재현율:0.3800,F1 스코어:0.2289
ROC 스코어: 0.6690, PR score : 0.0753
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2218   97]
 [  31   19]]
정확도: 0.9459, 정밀도 : 0.1638, 재현율:0.3800,F1 스코어:0.2289
ROC 스코어: 0.6690, PR score : 0.0753

In [19]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    # key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
    param_grid = {
      'criterion': ['entropy', 'gini'],
               'max_depth': [2,4,6,8],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [4,6,8,10],
               'min_samples_split': [4,6,8,10]}
    grid_search = GridSearchCV(tree
                            , param_grid = param_grid
                            , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                            , n_jobs = -1)  #-1: 모든 코어 사용

    grid_search.fit(x_train, y_train)

    # 최선의 결과 출력
    print('피처:{2} 최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_, feature))
    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
    dt_clf_best.fit(x_train, y_train)
    dt_best_pred = dt_clf_best.predict(x_test)
    dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

    print(feature,"디시전트리 파라미터적용") 
    get_clf_eval(y_test, dt_best_pred)
    get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


피처:mda 최적 하이퍼 파라미터 :{'criterion': 'gini', 'max_depth': 8, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 4}, 최적 평균 정확도 :0.955
mda 디시전트리 파라미터적용
오차행렬
[[2135  180]
 [  29   21]]
정확도: 0.9116, 정밀도 : 0.1045, 재현율:0.4200,F1 스코어:0.1673
ROC 스코어: 0.6711, PR score : 0.0561
임곗값: 0.1
오차행렬
[[2015  300]
 [  24   26]]
정확도: 0.8630, 정밀도 : 0.0798, 재현율:0.5200,F1 스코어:0.1383
ROC 스코어: 0.6952, PR score : 0.0516
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2039  276]
 [  26   24]]
정확도: 0.8723, 정밀도 : 0.0800, 재현율:0.4800,F1 스코어:0.1371
ROC 스코어: 0.6804, PR score : 0.0494
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2064  251]
 [  28   22]]
정확도: 0.8820, 정밀도 : 0.0806, 재현율:0.4400,F1 스코어:0.1362
ROC 스코어: 0.6658, PR score : 0.0473
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2064  251]
 [  28   22]]
정확도: 0.8820, 정밀도 : 0.0806, 재현율:0.4400,F1 스코어:0.1362
ROC 스코어: 0.6658, PR score : 0.0473
-------------------------------

In [20]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    rf_clf.fit(x_train, y_train)
    rf_pred = rf_clf.predict(x_test)
    rf_pred_proba = rf_clf.predict_proba(x_test)
    print(feature,"랜덤포레스트") 
    get_clf_eval(y_test, rf_pred)
    get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)
    


mda 랜덤포레스트
오차행렬
[[2279   36]
 [  34   16]]
정확도: 0.9704, 정밀도 : 0.3077, 재현율:0.3200,F1 스코어:0.3137
ROC 스코어: 0.6522, PR score : 0.1128
임곗값: 0.1
오차행렬
[[2020  295]
 [  18   32]]
정확도: 0.8677, 정밀도 : 0.0979, 재현율:0.6400,F1 스코어:0.1698
ROC 스코어: 0.7563, PR score : 0.0702
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2147  168]
 [  25   25]]
정확도: 0.9184, 정밀도 : 0.1295, 재현율:0.5000,F1 스코어:0.2058
ROC 스코어: 0.7137, PR score : 0.0753
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2222   93]
 [  30   20]]
정확도: 0.9480, 정밀도 : 0.1770, 재현율:0.4000,F1 스코어:0.2454
ROC 스코어: 0.6799, PR score : 0.0835
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2255   60]
 [  32   18]]
정확도: 0.9611, 정밀도 : 0.2308, 재현율:0.3600,F1 스코어:0.2812
ROC 스코어: 0.6670, PR score : 0.0966
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2279   36]
 [  34   16]]
정확도: 0.9704, 정밀도 : 0.3077, 재현율:0.3200,F1 스코어:0.3137
ROC 스코어: 0.6522, PR score : 0.112

In [22]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    grid_search = {'criterion': ['entropy', 'gini'],
                'max_depth': [2,4,6,8],
                'max_features': ['auto', 'sqrt'],
                'min_samples_leaf': [4,6,8,10],
                'min_samples_split': [4,6,8,10],
                'n_estimators': [20,40,60,80,100]}

    model = RandomForestClassifier(random_state=52)
    grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                                cv = 5,verbose=0, n_jobs = -1)
    grid_search.fit(x_train,y_train)

    # 최선의 결과 출력
    print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
    rf_clf_best.fit(x_train, y_train)
    rf_best_pred = rf_clf_best.predict(x_test)
    rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

    print(feature,"랜덤포레스트 파라미터적용") 
    get_clf_eval(y_test, rf_best_pred)
    get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


KeyboardInterrupt: 

In [23]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    lr_clf.fit(x_train, y_train)
    lr_pred = lr_clf.predict(x_test)
    lr_pred_proba = lr_clf.predict_proba(x_test)
    print(feature,"로지스틱회귀") 
    get_clf_eval(y_test, lr_pred)
    get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

mda 로지스틱회귀
오차행렬
[[2058  257]
 [  17   33]]
정확도: 0.8841, 정밀도 : 0.1138, 재현율:0.6600,F1 스코어:0.1941
ROC 스코어: 0.7745, PR score : 0.0823
임곗값: 0.1
오차행렬
[[1415  900]
 [  11   39]]
정확도: 0.6148, 정밀도 : 0.0415, 재현율:0.7800,F1 스코어:0.0789
ROC 스코어: 0.6956, PR score : 0.0370
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1738  577]
 [  14   36]]
정확도: 0.7501, 정밀도 : 0.0587, 재현율:0.7200,F1 스코어:0.1086
ROC 스코어: 0.7354, PR score : 0.0482
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1898  417]
 [  15   35]]
정확도: 0.8173, 정밀도 : 0.0774, 재현율:0.7000,F1 스코어:0.1394
ROC 스코어: 0.7599, PR score : 0.0605
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1987  328]
 [  16   34]]
정확도: 0.8545, 정밀도 : 0.0939, 재현율:0.6800,F1 스코어:0.1650
ROC 스코어: 0.7692, PR score : 0.0706
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2058  257]
 [  17   33]]
정확도: 0.8841, 정밀도 : 0.1138, 재현율:0.6600,F1 스코어:0.1941
ROC 스코어: 0.7745, PR score : 0.082

In [24]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    params = {'penalty': ['l2','l1','elasticnet','None'],
            'C':[0.01,0.1,1,3,5,10]}

    grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
    grid_search.fit(x_train,y_train)
    print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
    lr_clf_best.fit(x_train, y_train)
    lr_best_pred = lr_clf_best.predict(x_test)
    lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

    print(feature,"로지스틱회귀 파라미터적용") 
    get_clf_eval(y_test, lr_best_pred)
    get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미어:{'C': 10, 'penalty': 'l2'}, 최적 평균 정확도:0.901
mda 로지스틱회귀 파라미터적용
오차행렬
[[2056  259]
 [  17   33]]
정확도: 0.8833, 정밀도 : 0.1130, 재현율:0.6600,F1 스코어:0.1930
ROC 스코어: 0.7741, PR score : 0.0818
임곗값: 0.1
오차행렬
[[1420  895]
 [  12   38]]
정확도: 0.6165, 정밀도 : 0.0407, 재현율:0.7600,F1 스코어:0.0773
ROC 스코어: 0.6867, PR score : 0.0360
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1738  577]
 [  14   36]]
정확도: 0.7501, 정밀도 : 0.0587, 재현율:0.7200,F1 스코어:0.1086
ROC 스코어: 0.7354, PR score : 0.0482
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1900  415]
 [  15   35]]
정확도: 0.8182, 정밀도 : 0.0778, 재현율:0.7000,F1 스코어:0.1400
ROC 스코어: 0.7604, PR score : 0.0608
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1991  324]
 [  16   34]]
정확도: 0.8562, 정밀도 : 0.0950, 재현율:0.6800,F1 스코어:0.1667
ROC 스코어: 0.7700, PR score : 0.0713
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2056  259]
 [  17   33]]
정확도: 0.8833, 정밀도 : 0.1

In [25]:
# 1-5 KNN 
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):

    knn_clf.fit(x_train, y_train)
    knn_pred = knn_clf.predict(x_test)
    knn_pred_proba = knn_clf.predict_proba(x_test)
    print(feature,"KNN") 
    get_clf_eval(y_test, knn_pred)
    get_eval_by_threshold(y_test , knn_pred_proba[:,1].reshape(-1,1), thresholds)



mda KNN
오차행렬
[[2164  151]
 [  26   24]]
정확도: 0.9252, 정밀도 : 0.1371, 재현율:0.4800,F1 스코어:0.2133
ROC 스코어: 0.7074, PR score : 0.0768
임곗값: 0.1
오차행렬
[[2089  226]
 [  23   27]]
정확도: 0.8947, 정밀도 : 0.1067, 재현율:0.5400,F1 스코어:0.1782
ROC 스코어: 0.7212, PR score : 0.0674
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2140  175]
 [  26   24]]
정확도: 0.9150, 정밀도 : 0.1206, 재현율:0.4800,F1 스코어:0.1928
ROC 스코어: 0.7022, PR score : 0.0689
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2140  175]
 [  26   24]]
정확도: 0.9150, 정밀도 : 0.1206, 재현율:0.4800,F1 스코어:0.1928
ROC 스코어: 0.7022, PR score : 0.0689
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2164  151]
 [  26   24]]
정확도: 0.9252, 정밀도 : 0.1371, 재현율:0.4800,F1 스코어:0.2133
ROC 스코어: 0.7074, PR score : 0.0768
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2164  151]
 [  26   24]]
정확도: 0.9252, 정밀도 : 0.1371, 재현율:0.4800,F1 스코어:0.2133
ROC 스코어: 0.7074, PR score : 0.0768
-

In [26]:
# knn 파라미터 구하기
from sklearn.metrics import accuracy_score
acc = []
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    for k in range(1,20):
        knn_clf = KNeighborsClassifier(n_neighbors=k)
        knn_clf.fit(x_train, y_train)
        knn_pred = knn_clf.predict(x_test)
        knn_score=accuracy_score(y_test,knn_pred)

        acc.append(knn_score)
    int = acc.index(max(acc))+1
    print("최적거리")
    print(int)

    knn_clf_best =KNeighborsClassifier(n_neighbors=int)
    knn_clf_best.fit(x_train, y_train)
    knn_best_pred = knn_clf_best.predict(x_test)
    knn_best_pred_proba = knn_clf_best.predict_proba(x_test)   

    print(feature,"KNN 파라미터적용") 
    get_clf_eval(y_test, knn_best_pred)
    get_eval_by_threshold(y_test ,knn_best_pred_proba[:,1].reshape(-1,1), thresholds)      


최적거리
2
mda KNN 파라미터적용
오차행렬
[[2241   74]
 [  38   12]]
정확도: 0.9526, 정밀도 : 0.1395, 재현율:0.2400,F1 스코어:0.1765
ROC 스코어: 0.6040, PR score : 0.0496
임곗값: 0.1
오차행렬
[[2173  142]
 [  29   21]]
정확도: 0.9277, 정밀도 : 0.1288, 재현율:0.4200,F1 스코어:0.1972
ROC 스코어: 0.6793, PR score : 0.0664
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2173  142]
 [  29   21]]
정확도: 0.9277, 정밀도 : 0.1288, 재현율:0.4200,F1 스코어:0.1972
ROC 스코어: 0.6793, PR score : 0.0664
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2173  142]
 [  29   21]]
정확도: 0.9277, 정밀도 : 0.1288, 재현율:0.4200,F1 스코어:0.1972
ROC 스코어: 0.6793, PR score : 0.0664
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2173  142]
 [  29   21]]
정확도: 0.9277, 정밀도 : 0.1288, 재현율:0.4200,F1 스코어:0.1972
ROC 스코어: 0.6793, PR score : 0.0664
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2241   74]
 [  38   12]]
정확도: 0.9526, 정밀도 : 0.1395, 재현율:0.2400,F1 스코어:0.1765
ROC 스코어: 0.6040, PR sc

In [27]:
# 1-5 xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance 

xgb_wrapper = XGBClassifier()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):

    xgb_wrapper.fit(x_train, y_train, verbose = True)
    xgb_wrapper.fit(x_train, y_train)
    xgb_pred = xgb_wrapper.predict(x_test)
    xgb_pred_proba = xgb_wrapper.predict_proba(x_test)
    print(feature,"XGB") 
    get_clf_eval(y_test, xgb_pred)
    get_eval_by_threshold(y_test , xgb_pred_proba[:,1].reshape(-1,1), thresholds)

mda XGB
오차행렬
[[2257   58]
 [  32   18]]
정확도: 0.9619, 정밀도 : 0.2368, 재현율:0.3600,F1 스코어:0.2857
ROC 스코어: 0.6675, PR score : 0.0988
임곗값: 0.1
오차행렬
[[2180  135]
 [  27   23]]
정확도: 0.9315, 정밀도 : 0.1456, 재현율:0.4600,F1 스코어:0.2212
ROC 스코어: 0.7008, PR score : 0.0784
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2218   97]
 [  29   21]]
정확도: 0.9467, 정밀도 : 0.1780, 재현율:0.4200,F1 스코어:0.2500
ROC 스코어: 0.6890, PR score : 0.0870
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2233   82]
 [  31   19]]
정확도: 0.9522, 정밀도 : 0.1881, 재현율:0.3800,F1 스코어:0.2517
ROC 스코어: 0.6723, PR score : 0.0846
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2243   72]
 [  32   18]]
정확도: 0.9560, 정밀도 : 0.2000, 재현율:0.3600,F1 스코어:0.2571
ROC 스코어: 0.6644, PR score : 0.0855
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2257   58]
 [  32   18]]
정확도: 0.9619, 정밀도 : 0.2368, 재현율:0.3600,F1 스코어:0.2857
ROC 스코어: 0.6675, PR score : 0.0988
-

In [28]:
values[1]

'l2'

In [29]:
#xgb 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

xgb_wrapper
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    params = {
            'n_estimators':[40,43,46,50],
            'learning_rate':[0.03,0.05,0.04],
            'max_depth' : [0.8,1,2,3,4]
            }

    grid_clf = GridSearchCV(xgb_wrapper, param_grid=params, scoring='accuracy', cv=5)
    grid_clf.fit(x_train,y_train)
    print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_clf.best_params_, grid_clf.best_score_))

    a = grid_clf.best_params_
    keys=list(a.keys())
    values = list(a.values())
    xgb_clf_best = XGBClassifier( learning_rate=values[0],max_depth=values[1],n_estimators= values[2])
    xgb_clf_best.fit(x_train, y_train)
    xgb_best_pred = xgb_clf_best.predict(x_test)
    xgb_best_pred_proba = xgb_clf_best.predict_proba(x_test)   

    print(feature,"XGB 파라미터적용") 
    get_clf_eval(y_test, xgb_best_pred)
    get_eval_by_threshold(y_test ,xgb_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미어:{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 50}, 최적 평균 정확도:0.953
mda XGB 파라미터적용
오차행렬
[[2092  223]
 [  26   24]]
정확도: 0.8947, 정밀도 : 0.0972, 재현율:0.4800,F1 스코어:0.1616
ROC 스코어: 0.6918, PR score : 0.0576
임곗값: 0.1
오차행렬
[[1771  544]
 [  10   40]]
정확도: 0.7658, 정밀도 : 0.0685, 재현율:0.8000,F1 스코어:0.1262
ROC 스코어: 0.7825, PR score : 0.0590
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1909  406]
 [  13   37]]
정확도: 0.8228, 정밀도 : 0.0835, 재현율:0.7400,F1 스코어:0.1501
ROC 스코어: 0.7823, PR score : 0.0673
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1991  324]
 [  18   32]]
정확도: 0.8554, 정밀도 : 0.0899, 재현율:0.6400,F1 스코어:0.1576
ROC 스코어: 0.7500, PR score : 0.0651
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2029  286]
 [  22   28]]
정확도: 0.8698, 정밀도 : 0.0892, 재현율:0.5600,F1 스코어:0.1538
ROC 스코어: 0.7182, PR score : 0.0592
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2092  223]
 [  26